In [9]:
from envSetup import load_tag_pos
from AprilLib import TagDetect
import cv2
import numpy as np

img_index = 209

intrinsic = np.array(([207.9138,0,315.9991],[0,206.5876,242.7096],[0,0,1]),dtype=np.double)

distortion = np.array([0.2345,-0.1114,0,0],dtype=np.double)

tag_poses, centers, screens = load_tag_pos()
tg = TagDetect(tag_poses, intrinsic, distortion)
img = cv2.imread(f"../data/head3/{img_index}.jpg")

In [10]:
tags = tg.detect(img)

In [11]:
tags

[Detection(tag_family=b'tag36h11', tag_id=19, hamming=0, goodness=0.0, decision_margin=86.57315063476562, homography=array([[-1.68200611e-01, -4.01391508e-02, -1.28644011e+01],
        [ 3.82496361e-01, -7.11320599e-01, -7.10054105e+00],
        [ 1.49085918e-03, -1.04364095e-04, -3.49869614e-02]]), center=array([367.69129478, 202.94820606]), corners=array([[347.94772339, 186.17193604],
        [389.09213257, 179.88653564],
        [389.06427002, 221.10899353],
        [348.15689087, 223.99859619]]))]

In [12]:
for tag in tags:
    _, rvec, tvec = cv2.solvePnP(tg.tagStandard[str(tag.tag_id)], tag.corners, tg.intrinsic, tg.distortion)
    R, _ = cv2.Rodrigues(rvec)
    extrinsic_matrix = np.concatenate((R, tvec), axis=1)
    point2d, _ = cv2.projectPoints(screens[tag.tag_id], R, tvec, intrinsic, distortion)
    

In [13]:
point2d[:,0,:]

array([[258.29295215, 225.89604662],
       [291.7329157 , 223.96125583],
       [292.32154725, 293.13804908],
       [258.3145882 , 291.06267701]])

In [14]:
#cv2.polylines(img, point2d.astype(int), isClosed=True, color=(0, 255, 0), thickness=2)
#cv2.imwrite("test.jpg", img)

In [15]:
pts1 = point2d.astype(np.float32)

# 定义目标矩形的四个顶点坐标
pts2 = np.array([[0, 0], [27, 0], [27, 27], [0, 27]], dtype=np.float32)

# 计算透视变换矩阵
M = cv2.getPerspectiveTransform(pts1, pts2)

# 应用透视变换并拉伸为28x28的矩形
result = cv2.warpPerspective(img, M, (28, 28))

In [16]:
cv2.imwrite(f"get{img_index}.jpg", result)

True